\
            # 46. 前端框架：Vue.js 基础（Vue.js Fundamentals）

            目标：快速掌握 Vue 3 的核心使用方式：组件化、响应式、模板语法、事件与双向绑定、computed/watch。
本章示例默认使用 CDN 加载 Vue（需要网络）。若无法联网：可把 HTML 保存成本地文件用浏览器打开，或改用本地构建工具（Vite）。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- JS 基础（函数、对象、数组、事件、异步概念）
- HTML/CSS 基础


## 知识点地图

- 1. 为什么需要框架：状态驱动 UI 与组件化
- 2. Vue 3 最小应用：createApp + 模板
- 3. 模板语法速览：v-bind / v-on / v-model / v-if / v-for
- 4. computed 与 watch：派生状态 vs 副作用
- 5. 组件化：props / emits / slots（核心）
- 6. Composition API（了解）：setup、ref/reactive、生命周期
- 7. 工程化（了解）：Vite、SFC、Router、状态管理


## 自检清单（学完打勾）

- [ ] 理解组件化与单向数据流（props down / events up）
- [ ] 会写模板：v-bind/v-on/v-model/v-if/v-for
- [ ] 会用 computed 与 watch 解决派生状态与副作用
- [ ] 能把一个页面拆成多个组件并传递数据
- [ ] 知道 Vue 生态：Router/Pinia/Vite（概念）


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：为什么需要框架：状态驱动 UI 与组件化

传统手写 DOM 的问题：
- 状态分散在多个地方，变更容易遗漏
- 组件复用困难

框架核心思想：
- 维护“状态（state）”
- UI 由状态渲染（state -> view）
- 用户交互触发状态变化（event -> state）


## 知识点 2：Vue 3 最小应用：createApp + 模板

Vue 3 典型入口：
- `Vue.createApp(App).mount('#app')`
- App 里声明 data/methods（Options API）或 setup（Composition API）

下面用 Options API 写最小计数器（更直观）。


In [ ]:
from IPython.display import HTML, display

html = r"""
<div id="vue-demo-1" style="font-family:system-ui;border:1px solid #ddd;border-radius:12px;padding:10px;max-width:420px;">
  <h3 style="margin:0 0 10px 0;">Vue Counter</h3>
  <div id="app"></div>
</div>

<script src="https://unpkg.com/vue@3/dist/vue.global.prod.js"></script>
<script>
  const App = {
    data(){ return { count: 0 }; },
    template: `
      <div style="display:flex;align-items:center;gap:8px;">
        <button @click="count--">-</button>
        <b>{{ count }}</b>
        <button @click="count++">+</button>
      </div>
    `
  };

  try {
    Vue.createApp(App).mount('#app');
  } catch (e) {
    document.getElementById('app').textContent = 'Vue CDN load failed. Please check network, or open this HTML in a browser.';
  }
</script>
"""

display(HTML(html))


## 知识点 3：模板语法速览：v-bind / v-on / v-model / v-if / v-for

- `v-bind:attr`（缩写 `:attr`）：绑定属性
- `v-on:event`（缩写 `@event`）：绑定事件
- `v-model`：表单双向绑定（本质是 value + input 事件）
- `v-if/v-else`：条件渲染
- `v-for`：列表渲染（务必写 `:key`）

经验：
- 列表一定要 key，否则更新可能错位。
- v-model 适合表单；业务状态仍建议单向数据流。


## 知识点 4：computed 与 watch：派生状态 vs 副作用

- computed：根据状态“推导”出的值（缓存），例如过滤后的列表、总价。
- watch：监听变化做副作用（例如写入 localStorage、请求接口）。

原则：
- 能用 computed 就别用 watch（watch 更难推理）。


## 知识点 5：组件化：props / emits / slots（核心）

组件拆分的基本规则：
- 父组件通过 props 向下传数据
- 子组件通过 emits 向上抛事件
- slots 用于“插槽内容”（布局组件）

这套模式让数据流清晰、可测试、可维护。


## 知识点 6：Composition API（了解）：setup、ref/reactive、生命周期

Vue 3 更现代的写法：
- `setup()` 返回模板可用的变量/函数
- `ref()` 包装基本类型
- `reactive()` 包装对象
- 生命周期：onMounted/onUnmounted...

建议：先熟 Options API 再学 Composition API；理解响应式原理更重要。


## 知识点 7：工程化（了解）：Vite、SFC、Router、状态管理

- Vite：开发服务器快、打包现代
- SFC（.vue 单文件组件）：template/script/style 同文件（更工程化）
- Router：前端路由
- Pinia：状态管理

真实项目一般不会用 CDN 写应用，而是用 Vite 创建项目：
- `npm create vite@latest` -> 选择 Vue


## 常见坑

- v-for 不写 key：导致列表更新错位
- 把 watch 当 computed 用：副作用泛滥，难以维护
- 组件边界不清：一个组件太大、复用困难
- 滥用全局状态：任何地方都能改，难追踪
- 直接修改 props：破坏单向数据流（应该 emits 通知父组件改）


## 综合小案例：Vue Todo：过滤 + 统计 + localStorage 持久化（CDN 版）

实现要点：
- 输入框 v-model
- 列表 v-for + key
- computed：filteredTodos、remainCount
- watch：保存到 localStorage


In [ ]:
from IPython.display import HTML, display

html = r"""
<div style="font-family:system-ui;border:1px solid #ddd;border-radius:12px;padding:10px;max-width:560px;">
  <h3 style="margin:0 0 10px 0;">Vue Todo (CDN)</h3>
  <div id="app2"></div>
</div>

<script src="https://unpkg.com/vue@3/dist/vue.global.prod.js"></script>
<script>
  const KEY = 'vue.todo.v1';

  const App = {
    data(){
      let todos = [];
      try { todos = JSON.parse(localStorage.getItem(KEY) || '[]'); } catch(e) {}
      return { text:'', filter:'all', todos };
    },
    computed: {
      filtered(){
        if (this.filter === 'active') return this.todos.filter(t => !t.done);
        if (this.filter === 'done') return this.todos.filter(t => t.done);
        return this.todos;
      },
      remain(){ return this.todos.filter(t => !t.done).length; }
    },
    watch: {
      todos: {
        deep: true,
        handler(v){ localStorage.setItem(KEY, JSON.stringify(v)); }
      }
    },
    methods: {
      add(){
        const title = (this.text || '').trim();
        if (!title) return;
        this.todos.push({id: Date.now().toString(36), title, done:false});
        this.text = '';
      },
      del(id){ this.todos = this.todos.filter(t => t.id !== id); }
    },
    template: `
      <div style="display:flex;gap:6px;">
        <input v-model="text" placeholder="new todo" style="flex:1;padding:6px;" @keyup.enter="add" />
        <button @click="add">Add</button>
      </div>

      <div style="margin:10px 0;display:flex;gap:6px;align-items:center;">
        <span style="color:#555;">remain: <b>{{ remain }}</b></span>
        <button @click="filter='all'" :style="{fontWeight: filter==='all'?'700':'400'}">all</button>
        <button @click="filter='active'" :style="{fontWeight: filter==='active'?'700':'400'}">active</button>
        <button @click="filter='done'" :style="{fontWeight: filter==='done'?'700':'400'}">done</button>
      </div>

      <ul style="padding-left:18px;">
        <li v-for="t in filtered" :key="t.id" style="margin:6px 0;">
          <label style="cursor:pointer;">
            <input type="checkbox" v-model="t.done" />
            <span :style="{textDecoration: t.done?'line-through':'none', color: t.done?'#777':'#111'}">{{ t.title }}</span>
          </label>
          <button @click="del(t.id)" style="margin-left:8px;">Del</button>
        </li>
      </ul>
    `
  };

  try {
    Vue.createApp(App).mount('#app2');
  } catch (e) {
    document.getElementById('app2').textContent = 'Vue CDN load failed. Please check network, or copy this HTML to a local file and open it in a browser.';
  }
</script>
"""

display(HTML(html))


## 自测题（不写代码也能回答）

- props down / events up 解决了什么问题？
- computed 与 watch 的差异是什么？各自适合什么场景？
- 为什么 v-for 必须写 key？
- Vue 的响应式大致靠什么机制实现？（概念）


## 练习题（建议写代码）

- 把 Todo 拆成 2 个组件：TodoInput、TodoList（props + emits）。
- 为 Todo 增加“编辑模式”（双击编辑，回车保存）。
- 把 CDN 版本迁移为 Vite 项目结构（SFC）。
